In [1]:
import os
from openslide import OpenSlide, open_slide, deepzoom
import numpy as np
from scipy import linalg
import napari
from skimage.io import imread
from dask import delayed
import dask.array as da
#os.chdir("../..")


In [2]:
sm_path = "D:/repos/swe22_final/sm.svs"
#slide = OpenSlide(sm_path)

In [15]:
# https://github.com/qupath/qupath/wiki/Supported-image-formats
# https://programtalk.com/vs4/python/jlevy44/PathFlowAI/deprecated_work/build/lib/pathflowai/utils.py/
def svs2dask_array(svs_file, tile_size=1000, overlap=0, remove_last=True, allow_unknown_chunksizes=False):
	"""Convert SVS, TIF or TIFF to dask array.

	Parameters
	----------
	svs_file:str
		Image file.
	tile_size:int
		Size of chunk to be read in.
	overlap:int
		Do not modify, overlap between neighboring tiles.
	remove_last:bool
		Remove last tile because it has a custom size.
	allow_unknown_chunksizes: bool
		Allow different chunk sizes, more flexible, but slowdown.

	Returns
	-------
	dask.array
		Dask Array.

	>>> arr=svs2dask_array(svs_file, tile_size=1000, overlap=0, remove_last=True, allow_unknown_chunksizes=False)
	>>> arr2=arr.compute()
	>>> arr3=to_pil(cv2.resize(arr2, dsize=(1440,700), interpolation=cv2.INTER_CUBIC))
	>>> arr3.save(test_image_name)"""
	img=open_slide(svs_file)
	gen=deepzoom.DeepZoomGenerator(img, tile_size=tile_size, overlap=overlap, limit_bounds=True)
	
	#for dim in range(len(gen.level_dimensions)-1):
	max_level = len(gen.level_dimensions)-1
	#max_level = dim #len(gen.level_dimensions)-1
	n_tiles_x, n_tiles_y = gen.level_tiles[max_level]
	get_tile = lambda i,j: np.array(gen.get_tile(max_level,(i,j))).transpose((1,0,2))
	sample_tile = get_tile(0,0)
	sample_tile_shape = sample_tile.shape
	dask_get_tile = delayed(get_tile, pure=True)
	arr=da.concatenate([da.concatenate([da.from_delayed(dask_get_tile(i,j),sample_tile_shape,np.uint8) for j in range(n_tiles_y - (0 if not remove_last else 1))],allow_unknown_chunksizes=allow_unknown_chunksizes,axis=1) for i in range(n_tiles_x - (0 if not remove_last else 1))],allow_unknown_chunksizes=allow_unknown_chunksizes)#.transpose([1,0,2])
	
	return arr

In [ ]:
lvls = deepzoom.DeepZoomGenerator(open_slide(sm_path), tile_size=1000, overlap=0, limit_bounds=True).level_count

In [13]:
viewer = napari.Viewer()
viewer.add_image( svs2dask_array(sm_path), contrast_limits=[0,255])

TypeError: 'int' object is not iterable

In [ ]:

deepzoom.DeepZoomGenerator(open_slide(sm_path), tile_size=1000, overlap=0, limit_bounds=True).level_count

In [90]:
# https://github.com/qupath/qupath/wiki/Supported-image-formats
# https://programtalk.com/vs4/python/jlevy44/PathFlowAI/deprecated_work/build/lib/pathflowai/utils.py/
def svs2dask_array_pyramid(svs_file, level, tile_size=1000, overlap=0, remove_last=True, allow_unknown_chunksizes=False):
	"""Convert SVS, TIF or TIFF to dask array.

	Parameters
	----------
	svs_file:str
		Image file.
	tile_size:int
		Size of chunk to be read in.
	overlap:int
		Do not modify, overlap between neighboring tiles.
	remove_last:bool
		Remove last tile because it has a custom size.
	allow_unknown_chunksizes: bool
		Allow different chunk sizes, more flexible, but slowdown.

	Returns
	-------
	dask.array
		Dask Array.

	>>> arr=svs2dask_array(svs_file, tile_size=1000, overlap=0, remove_last=True, allow_unknown_chunksizes=False)
	>>> arr2=arr.compute()
	>>> arr3=to_pil(cv2.resize(arr2, dsize=(1440,700), interpolation=cv2.INTER_CUBIC))
	>>> arr3.save(test_image_name)"""
	img=open_slide(svs_file)
	gen=deepzoom.DeepZoomGenerator(img)
	
	max_level = len(gen.level_dimensions)-1
	n_tiles_x, n_tiles_y = gen.level_tiles[max_level]
	get_tile = lambda i,j: np.array(gen.get_tile(level,(i,j))).transpose((1,0,2))
	sample_tile = get_tile(0,0)
	sample_tile_shape = sample_tile.shape
	dask_get_tile = delayed(get_tile, pure=True)
	arr = (da.concatenate([da.concatenate([da.from_delayed(dask_get_tile(i,j),sample_tile_shape,np.uint8) for j in range(n_tiles_y - (0 if not remove_last else 1))],allow_unknown_chunksizes=allow_unknown_chunksizes,axis=1) for i in range(n_tiles_x - (0 if not remove_last else 1))],allow_unknown_chunksizes=allow_unknown_chunksizes))#.transpose([1,0,2]))

	return arr

In [85]:
lvls = deepzoom.DeepZoomGenerator(open_slide(sm_path), tile_size=1000, overlap=0, limit_bounds=True).level_count
daskPy= svs2dask_array_pyramid(sm_path, 1)


In [86]:
daskPy

,Array,Chunk
Bytes,3.66 kiB,3 B
Shape,"(32, 39, 3)","(1, 1, 3)"
Count,4992 Tasks,1248 Chunks
Type,uint8,numpy.ndarray


In [91]:
lvls = deepzoom.DeepZoomGenerator(open_slide(sm_path)).level_count
viewer = napari.Viewer() 
viewer.add_image( [svs2dask_array_pyramid(sm_path, level) for level in range(lvls)], contrast_limits=[0,255], multiscale = True)

MemoryError: Unable to allocate 3.61 GiB for an array with shape (32767, 39423, 3) and data type uint8

ValueError: Shape of in dividual tiles in multiscale (32000, 39000, 3) cannot exceed GL_MAX_TEXTURE_SIZE 32768. Rendering is currently in 2D mode.

In [ ]:
img=open_slide(sm_path)
gen=deepzoom.DeepZoomGenerator(img, tile_size=1000, overlap=0, limit_bounds=True)
gen.level_dimensions

In [ ]:
def getHEDfromRGB():
    
    # Conversion matrix for RGB-HED
    
    # Haematoxylin-Eosin-DAB colorspace
    # From original Ruifrok's paper: A. C. Ruifrok and D. A. Johnston,
    # "Quantification of histochemical staining by color deconvolution,"
    # Analytical and quantitative cytology and histology / the International
    # Academy of Cytology [and] American Society of Cytology, vol. 23, no. 4,
    # pp. 291-9, Aug. 2001.
    rgb_from_hed = np.array([[0.65, 0.70, 0.29],
                            [0.07, 0.99, 0.11],
                            [0.27, 0.57, 0.78]])
    hed_from_rgb = linalg.inv(rgb_from_hed)

    return


In [ ]:
viewer = napari.Viewer()
#t, c, z, y_slice, x_slice

In [ ]:
viewer.add_image( img.reader.dask_data.rechunk() , contrast_limits=[0,255])

loci.formats.FormatException: loci.formats.FormatException: Image plane too large. Only 2GB of data can be extracted at one time. You can work around the problem by opening the plane in tiles; for further details, see: https://docs.openmicroscopy.org/bio-formats/6.10.0-SNAPSHOT/about/bug-reporting.html#common-issues-to-check

In [ ]:
img = AICSImage("sm.svs") #aicsimageio.readers.bioformats_reader.BioFile("sm.svs")
data = da.array.rechunk(img.get_image_dask_data)
data

d:\Software\Conda\lib\site-packages\ome_types\_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


dask.array<rechunk-merge, shape=(39328, 32639), dtype=uint8, chunksize=(9832, 13650), chunktype=numpy.ndarray>

In [ ]:
data.commpute_chunk_sizes

NameError: name 'commpute_chunk_sizes' is not defined

In [ ]:
img.get_image_dask_data("CXY")

dask.array<transpose, shape=(1, 32639, 39328), dtype=uint8, chunksize=(1, 32639, 39328), chunktype=numpy.ndarray>

In [ ]:
da.array.rechunk(img.xarray_dask_data)

,Array,Chunk
Bytes,3.59 GiB,127.99 MiB
Shape,"(1, 1, 1, 39328, 32639, 3)","(1, 1, 1, 4916, 27300, 1)"
Count,99 Tasks,48 Chunks
Type,uint8,xarray.core.dataarray.DataArray
